In [1]:
from __future__ import annotations

from torch.utils.data import random_split

from mmpfn.datasets.salary import SalaryDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn import MMPFNClassifier
from mmpfn.models.mmpfn.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_mmpfn

/home/wall/anaconda3/envs/mmpfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # for debugging

In [3]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/salary")
dataset = SalaryDataset(data_path)
_ = dataset.get_embeddings()

Load embeddings from embeddings/salary/salary.pt


In [4]:
accuracy_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    text_train = train_dataset.dataset.embeddings[train_dataset.indices]
    text_test = test_dataset.dataset.embeddings[test_dataset.indices]
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_salary.ckpt"

    fine_tune_mmpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=text_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM+CAP', # MGM MGM+CAP
        mgm_heads=8,
        cap_heads=4,
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='MGM+CAP', # MGM MGM+CAP
        mgm_heads=8,
        cap_heads=4,
    )

    clf_finetuned = model_finetuned.fit(X_train, text_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test, text_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:  13%|█▎        | 13/100 [00:18<02:15,  1.56s/it, Best Val. Loss=1.24, Best Val. Score=-1.24, Training Loss=1.32, Val. Loss=1.24, Patience=38, Utilization=0, Grad Norm=15.6][2025-09-17 22:44:31,641] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  38%|███▊      | 38/100 [00:55<01:31,  1.48s/it, Best Val. Loss=1.22, Best Val. Score=-1.22, Training Loss=1.25, Val. Loss=1.22, Patience=14, Utilization=0, Grad Norm=5.95][2025-09-17 22:45:08,192] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  78%|███████▊  | 78/100 [01:54<00:32,  1.48s/it, Best Val. Loss=1.21, Best Val. Score=-1.21, Training Loss=1.25, Val. Loss=1.21, Patience=-25, Utilization=0, Grad Norm=6.78][2025-09-17 22:46:07,599] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:27,  1.47s/it, Best Val. Loss=1.21, Best Val. Score=-1.21, Training Loss=1.24, Val. Loss=1.21, Patience=-47, Utilization=0, 

accuracy_score (Finetuned): 0.46458333333333335


Fine-tuning Steps:  10%|█         | 10/100 [00:13<02:13,  1.49s/it, Best Val. Loss=1.21, Best Val. Score=-1.21, Training Loss=1.21, Val. Loss=1.21, Patience=41, Utilization=0, Grad Norm=11.2][2025-09-17 22:46:56,456] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  19%|█▉        | 19/100 [00:26<02:00,  1.48s/it, Best Val. Loss=1.2, Best Val. Score=-1.2, Training Loss=1.24, Val. Loss=1.2, Patience=33, Utilization=0, Grad Norm=4.85]   [2025-09-17 22:47:09,286] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:24,  1.44s/it, Best Val. Loss=1.19, Best Val. Score=-1.19, Training Loss=1.17, Val. Loss=1.19, Patience=-48, Utilization=0, Grad Norm=6.57]                         
[2025-09-17 22:49:06,349] INFO - Initial Validation Loss: 1.279693744765703 Best Validation Loss: 1.1887137045973468 Total Steps: 101 Best Step: 81 Total Time Spent: 144.89250087738037


accuracy_score (Finetuned): 0.45416666666666666


Fine-tuning Steps:   6%|▌         | 6/100 [00:07<02:17,  1.46s/it, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.23, Val. Loss=1.28, Patience=45, Utilization=0, Grad Norm=5.73][2025-09-17 22:49:17,218] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   9%|▉         | 9/100 [00:11<02:05,  1.38s/it, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.26, Val. Loss=1.48, Patience=43, Utilization=0, Grad Norm=9.13][2025-09-17 22:49:21,151] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:24,  1.44s/it, Best Val. Loss=1.24, Best Val. Score=-1.24, Training Loss=1.14, Val. Loss=1.36, Patience=-48, Utilization=0, Grad Norm=5.76]                         
[2025-09-17 22:51:33,058] INFO - Initial Validation Loss: 1.3121701039054423 Best Validation Loss: 1.239178009383676 Total Steps: 101 Best Step: 25 Total Time Spent: 144.81754994392395


accuracy_score (Finetuned): 0.43020833333333336


Fine-tuning Steps:   3%|▎         | 3/100 [00:03<02:32,  1.57s/it, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.28, Val. Loss=1.25, Patience=48, Utilization=0, Grad Norm=12.1][2025-09-17 22:51:39,798] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   5%|▌         | 5/100 [00:05<02:11,  1.38s/it, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.29, Val. Loss=1.25, Patience=47, Utilization=0, Grad Norm=7.42][2025-09-17 22:51:42,319] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:25,  1.45s/it, Best Val. Loss=1.22, Best Val. Score=-1.22, Training Loss=1.14, Val. Loss=1.22, Patience=-48, Utilization=0, Grad Norm=4.32]                         
[2025-09-17 22:54:01,013] INFO - Initial Validation Loss: 1.3086846813533153 Best Validation Loss: 1.2231575049721817 Total Steps: 101 Best Step: 95 Total Time Spent: 146.00294375419617


accuracy_score (Finetuned): 0.4510416666666667


Fine-tuning Steps:   3%|▎         | 3/100 [00:02<02:24,  1.49s/it, Best Val. Loss=1.23, Best Val. Score=-1.23, Training Loss=1.3, Val. Loss=1.23, Patience=48, Utilization=0, Grad Norm=6.58] [2025-09-17 22:54:07,490] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  11%|█         | 11/100 [00:14<02:12,  1.49s/it, Best Val. Loss=1.22, Best Val. Score=-1.22, Training Loss=1.15, Val. Loss=1.22, Patience=41, Utilization=0, Grad Norm=4.55][2025-09-17 22:54:18,987] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  25%|██▌       | 25/100 [00:34<01:50,  1.47s/it, Best Val. Loss=1.2, Best Val. Score=-1.2, Training Loss=1.16, Val. Loss=1.2, Patience=28, Utilization=0, Grad Norm=4.49]   [2025-09-17 22:54:39,293] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [02:24,  1.45s/it, Best Val. Loss=1.19, Best Val. Score=-1.19, Training Loss=1.14, Val. Loss=1.2, Patience=-47, Utilization=0, Gra

accuracy_score (Finetuned): 0.4479166666666667


In [5]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.4495833333333333
Std Accuracy: 0.011191980015469406


In [6]:
(0.459375+0.46145833333333336+0.4552083333333333)/3

0.45868055555555554